# New Section: Social Security Fund Planning

In [1]:
#import all the necessary packages
import numpy as np #for numerical array data
import pandas as pd #for tabular data
from scipy.optimize import minimize
import matplotlib.pyplot as plt #for plotting purposes
%matplotlib inline
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

import cvxpy as cp
from sklearn import datasets
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

In [44]:
life_table = pd.read_csv('datasets/Life_expectancy.csv')
life_table.head()

Age  Probability_M  Number_M  Life_M  Probability_F  Number_F  Life_F
0    0       0.006383  100000.0   76.15       0.005374  100000.0   80.97
1    1       0.000453   99362.0   75.63       0.000353   99463.0   80.41
2    2       0.000282   99317.0   74.67       0.000231   99427.0   79.44
3    3       0.000230   99289.0   73.69       0.000165   99405.0   78.45
4    4       0.000169   99266.0   72.71       0.000129   99388.0   77.47

In [242]:
# Parameters:
def SS_contribution(tax, payment, sex, inf_cum, int_cum, COLA_cum, n_people=100000, curr_age=25, death_reduce=.75/100):
    years_togo = 65 - curr_age
    yearly_contribution = np.zeros(len(inflation_cum))
    people_alive = n_people
    
    if sex == 'M':
        for i in range(years_togo):
            survive_prob = 1 - life_table['Probability_M'][int(curr_age+i)] * ((1-death_reduce)**i)
            people_alive = people_alive * survive_prob
            yearly_contribution[i] = people_alive * tax * inf_cum[i]
            
        for i in range(65,110):
            survive_prob = 1 - life_table['Probability_M'][int(i)] * ((1-death_reduce)**(i-curr_age))
            lumpsum_payment = 255 * (1-survive_prob) * people_alive
            people_alive = people_alive * survive_prob
            yearly_contribution[i-curr_age] = (-people_alive * payment - lumpsum_payment) * COLA_cum[i-curr_age]
            
        return yearly_contribution / int_cum
    
    else:
        for i in range(years_togo):
            survive_prob = 1 - life_table['Probability_F'][int(curr_age+i)] * ((1-death_reduce)**i)
            people_alive = people_alive * survive_prob
            yearly_contribution[i] = people_alive * tax * inf_cum[i]
            
        for i in range(65,110):
            survive_prob = 1 - life_table['Probability_F'][int(i)] * ((1-death_reduce)**(i-curr_age))
            lumpsum_payment = 255 * (1-survive_prob) * people_alive
            people_alive = people_alive * survive_prob
            yearly_contribution[i-curr_age] = (-people_alive * payment - lumpsum_payment) * COLA_cum[i-curr_age]
            
        return yearly_contribution / int_cum

In [232]:
discount_values = pd.DataFrame(np.concatenate((inflation_cum, COLA_cum)))

discount_values.to_excel('discount_values2.xlsx')

In [229]:
inflation_vec = 1.021 + np.random.randn(85) * 0.01  # Largest possible age: 110
COLA_vec = 0.1023 + 0.9 * inflation_vec + np.random.randn(85) * 0.009 # COLA...
interest_vec = inflation_vec + 0.003

inflation_cum = np.cumprod(inflation_vec)
interest_cum = np.cumprod(interest_vec)  # Discount rate
COLA_cum = np.cumprod(COLA_vec)

In [230]:
np.mean(inflation_vec), np.std(inflation_vec), np.mean(COLA_vec), np.std(COLA_vec)

(1.021423768314162,
 0.010389794146922102,
 1.0220560638426077,
 0.013456968484907322)

In [250]:
test_pay = 2861*12
test_income = 132900
test_rate = 13/100
test_tax = test_income * test_rate
print(np.sum(SS_contribution(test_tax, test_pay, 'M', inflation_cum, interest_cum, COLA_cum,n_people=53167) / 1e9))
print(np.sum(SS_contribution(test_tax, test_pay, 'F', inflation_cum, interest_cum, COLA_cum,n_people=46833) / 1e9))

3.148915786401107
-1.9274243091731404
